# 将数据导入TensorFlow

注意：将数据导入到 TensorFlow 程序的首选方法是使用数据集 API。

另外还有三种方法可以将数据导入到 TensorFlow 程序中：

- <a href='#feeding'>Feeding</a>：Python的代码在运行每个步骤时提供数据。
- <a href='#load_file'>从文件读取</a>：输入管道从 TensorFlow 图的开始处读取文件中的数据。
- <a href='#pre_load'>预加载数据</a>：TensorFlow 图中的常量或变量保存所有数据（对于小型数据集）。

## <a name='feeding'>Feeding</a>

TensorFlow 的 feed 机制允许您在计算图中向任何张量注入数据。因此， python 计算可以直接将数据导入到图中。

通过 feed_dict 参数向启动计算的 run() 或 eval () 调用提供 feed 数据。

> 注意：“Feeding” 是将数据传送到 TensorFlow 程序的最有效的方式，只能用于小型实验和调试。

In [ ]:
import tensorflow as tf

with tf.Session() as sess:   
    input = tf.placeholder(tf.float32)   
    classifier = ...   
    print(classifier.eval(feed_dict={input: my_python_preprocessing_fn()}))  

虽然可以使用 Feed 数据（包括变量和常量）替换任何 Tensor，但最佳做法是使用 tf.placeholder 节点。

placeholder（占位符）只是作为 feed 的目标存在。  
它未初始化，不包含任何数据如果占位符在没有 Feed 的情况下执行，则会产生错误，因此您不会忘记将其遗忘。

在 tensorflow/examples/tutorials/mnist/fully_connected_feed.py 中可以找到在 MNIST 数据上使用占位符和 Feeding 训练的示例, 并在 MNIST 教程中进行了说明。

In [2]:
import tensorflow as tf

input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)
output = tf.multiply(input1, input2)

with tf.Session() as sess:
    print(sess.run([output], feed_dict={input1:[7.], input2:[2.] }))

[array([14.], dtype=float32)]


---

## <a name='load_file'>从文件导入</a>

从文件导入记录的典型管道有以下几个阶段：
- 文件名列表
- 可选文件名洗牌
- 可选时期限制
- 文件名队列
- 用于文件格式的读取器
- 读者用于读取记录的解码器
- 可选预处理
- 示例队列

> 注意：本节讨论使用基于队列的API实现输入管道，该 API 可以被 \${\$datasets$Dataset API\} 完整地替换。

### 文件名，shuffling 和 epoch 限制

对于文件名列表，请使用常量字符串张量。

如：
- ["file0", "file1"]或[("file%d" % i) for i in range(2)]
- 或函数：tf.train.match_filenames_once。



将文件名列表传递给 tf.train.string_input_producer 函数。  

string_input_producer 创建一个 FIFO 队列，用于保存文件名，直到读取器需要它们为止。

string_input_producer 有选择的 shuffling 和设置一个最大的 epoch 数。

队列运行程序为每个 epoch 将文件名的整个列表添加到队列中一次，如果 shuffling = True，则在一个 epoch 中重新排列文件名。此过程提供了一个统一的文件取样，以便相对于彼此不会对示例进行低估或过度采样。

队列运行程序在与从队列中抽取文件名的读取器分开的线程中工作，因此，shuffling 和  enqueuing 进程不会阻止读取器。

### 文件格式

选择与您的输入文件格式相匹配的读取器，并将文件名队列传递给读取器的读取方法。

read 方法输出一个标识文件和记录的密钥 (如果有一些奇怪的记录，则对调试有用) 和一个标量字符串值。使用一个 (或多个) 解码器和转换 ops 将此字符串解码为构成示例的张量。

#### 1. CSV文件

若要以逗号分隔值 (CSV) 格式读取文本文件, 请使用 tf.TextLineReader 与 tf.decode_csv 操作。

例如：

In [32]:
import tensorflow as tf

# string_input_producer 创建一个 FIFO 队列，用于保存文件名，直到读取器需要它们为止。
filename_queue = tf.train.string_input_producer(['./tf_file.csv'])

# 使用 tf.TextLineReader 读取csv文件
# 每次一行
reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

# 指定数据类型：record_defaults里面的数据类型决定了读取的数据类型，而且必须是list形式
# 指定默认值：如果读取的值为空，则按record_defaults中对应的值作为 默认值
record_defaults = [['1.0'], ['1.0'], ['1.0']]

# 解析出的每一个属性都是rank为0的标量
col1, col2, col3 = tf.decode_csv(value, record_defaults=record_defaults)
# tf.stack是拼接，此处是上下拼接
features = tf.stack([col1, col2, col3])


with tf.Session() as sess: 
    # 线程协调器
    coord = tf.train.Coordinator()
    # 启动线程
    threads = tf.train.start_queue_runners(coord=coord)

    for i in range(10):
        # 提取一个instance
        example, label= sess.run([features, col1])

        print(label, example)

    #循环结束后，请求关闭所有线程
    coord.request_stop()
    coord.join(threads)

b'col1' [b'col1' b'col2' b'col3']
b'1' [b'1' b'1' b'0']
b'2' [b'2' b'4' b'0.602059991']
b'3' [b'3' b'9' b'0.954242509']
b'4' [b'4' b'16' b'1.204119983']
b'5' [b'5' b'25' b'1.397940009']
b'6' [b'6' b'36' b'1.556302501']
b'7' [b'7' b'49' b'1.69019608']
b'8' [b'8' b'64' b'1.806179974']
b'9' [b'9' b'81' b'1.908485019']


每次读取的执行都从文件中读取一行。   
然后，decode_csv 操作将结果解析为张量列表。   
该 record_defaults 参数确定生成的张量的类型，并设置在输入字符串中缺少值时要使用的默认值。

在调用 run 或 eval 执行读取之前，必须调用 tf.train.start_queue_runners 来填充队列。否则，读取将在等待队列中的文件名时阻止。



#### 2. 固定长度记录

[array([b'col1', b'col2', b'col3'], dtype=object)]
[array([b'1', b'1', b'0'], dtype=object)]
[array([b'2', b'4', b'0.602059991'], dtype=object)]
[array([b'3', b'9', b'0.954242509'], dtype=object)]
[array([b'4', b'16', b'1.204119983'], dtype=object)]
[array([b'5', b'25', b'1.397940009'], dtype=object)]
[array([b'6', b'36', b'1.556302501'], dtype=object)]
[array([b'7', b'49', b'1.69019608'], dtype=object)]
[array([b'8', b'64', b'1.806179974'], dtype=object)]
[array([b'9', b'81', b'1.908485019'], dtype=object)]


b'col1' [b'col1' b'col2' b'col3']
b'1' [b'1' b'1' b'0']
b'2' [b'2' b'4' b'0.602059991']
b'3' [b'3' b'9' b'0.954242509']
b'4' [b'4' b'16' b'1.204119983']
b'5' [b'5' b'25' b'1.397940009']
b'6' [b'6' b'36' b'1.556302501']
b'7' [b'7' b'49' b'1.69019608']
b'8' [b'8' b'64' b'1.806179974']
b'9' [b'9' b'81' b'1.908485019']


In [24]:
import tensorflow as tf

a = tf.constant([1,2,3])
b = tf.constant([4,5,6])
c = tf.stack([a,b])

with tf.Session() as sess:
    print(sess.run(c))

[[1 2 3]
 [4 5 6]]
